In [1]:
import mrsqm
import numpy as np
import pandas as pd
from sklearn import metrics
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import timeit

In [2]:
# code to read arff data
# Source: https://github.com/alan-turing-institute/sktime/blob/main/sktime/utils/data_io.py

def load_from_arff_to_dataframe(

    full_file_path_and_name,
    has_class_labels=True,
    return_separate_X_and_y=True,
    replace_missing_vals_with="NaN",
):
    """Load data from a .ts file into a Pandas DataFrame.
    Parameters
    ----------
    full_file_path_and_name: str
        The full pathname of the .ts file to read.
    has_class_labels: bool
        true then line contains separated strings and class value contains
        list of separated strings, check for 'return_separate_X_and_y'
        false otherwise.
    return_separate_X_and_y: bool
        true then X and Y values should be returned as separate Data Frames (
        X) and a numpy array (y), false otherwise.
        This is only relevant for data.
    replace_missing_vals_with: str
       The value that missing values in the text file should be replaced
       with prior to parsing.
    Returns
    -------
    DataFrame, ndarray
        If return_separate_X_and_y then a tuple containing a DataFrame and a
        numpy array containing the relevant time-series and corresponding
        class values.
    DataFrame
        If not return_separate_X_and_y then a single DataFrame containing
        all time-series and (if relevant) a column "class_vals" the
        associated class values.
    """
    
    instance_list = []
    class_val_list = []

    data_started = False
    is_multi_variate = False
    is_first_case = True

    # Parse the file
    # print(full_file_path_and_name)
    with open(full_file_path_and_name, "r", encoding="utf-8") as f:
        for line in f:

            if line.strip():
                if (
                    is_multi_variate is False
                    and "@attribute" in line.lower()
                    and "relational" in line.lower()
                ):
                    is_multi_variate = True

                if "@data" in line.lower():
                    data_started = True
                    continue

                # if the 'data tag has been found, the header information
                # has been cleared and now data can be loaded
                if data_started:
                    line = line.replace("?", replace_missing_vals_with)

                    if is_multi_variate:
                        if has_class_labels:
                            line, class_val = line.split("',")
                            class_val_list.append(class_val.strip())
                        dimensions = line.split("\\n")
                        dimensions[0] = dimensions[0].replace("'", "")

                        if is_first_case:
                            for _d in range(len(dimensions)):
                                instance_list.append([])
                            is_first_case = False

                        for dim in range(len(dimensions)):
                            instance_list[dim].append(
                                pd.Series(
                                    [float(i) for i in dimensions[dim].split(",")]
                                )
                            )

                    else:
                        if is_first_case:
                            instance_list.append([])
                            is_first_case = False

                        line_parts = line.split(",")
                        if has_class_labels:
                            instance_list[0].append(
                                pd.Series(
                                    [
                                        float(i)
                                        for i in line_parts[: len(line_parts) - 1]
                                    ]
                                )
                            )
                            class_val_list.append(line_parts[-1].strip())
                        else:
                            instance_list[0].append(
                                pd.Series(
                                    [float(i) for i in line_parts[: len(line_parts)]]
                                )
                            )

    x_data = pd.DataFrame(dtype=np.float32)
    for dim in range(len(instance_list)):
        x_data["dim_" + str(dim)] = instance_list[dim]

    if has_class_labels:
        if return_separate_X_and_y:
            return x_data, np.asarray(class_val_list)
        else:
            x_data["class_vals"] = pd.Series(class_val_list)

    return x_data

# code to visualize saliency map
# Source: https://github.com/mlgig/explanation4tsc/blob/master/3.%20Compare%20Metrics%20and%20Draw%20Figures.ipynb
def plot_time_series_with_color(ts, weight, save = False):   
    cas = weight
    
    def transform(X):
        ma,mi = np.max(X), np.min(X)
        X = (X - mi)/(ma-mi)
        return X*100
    cas = transform(cas)

    max_length1, max_length2 = len(weight),10000 #
    x1 = np.linspace(0,max_length1,num = max_length1)
    x2 = np.linspace(0,max_length1,num = max_length2)
    y1 = ts
    f = interp1d(x1, y1)

    fcas = interp1d(x1, cas)
    cas = fcas(x2)

    plt.figure(figsize = (5,3.5))
    plt.scatter(x2,f(x2), c = cas, cmap = 'jet', marker='.', s= 1,vmin=0,vmax = 100)    
    plt.colorbar()
    if save: plt.savefig('imgout/SM_%s_%s_Class%d_idx%d.png' %(dataset, explanation_method, label[i],i))


In [3]:
dataset="Coffee"
X_train,y_train = load_from_arff_to_dataframe("data/" + dataset + "/" + dataset + "_TRAIN.arff")
X_test,y_test = load_from_arff_to_dataframe("data/" + dataset + "/" + dataset + "_TEST.arff")

print(dataset)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

X_train

Coffee
(28, 1) (28,)
(28, 1) (28,)


,dim_0
0,0 -0.518419 1 -0.485884 2 -0.50500...
1,0 -0.548462 1 -0.533681 2 -0.51472...
2,0 -0.472634 1 -0.415546 2 -0.35992...
3,0 -0.509521 1 -0.484218 2 -0.47795...
4,0 -0.563427 1 -0.533896 2 -0.54382...
5,0 -0.441485 1 -0.449366 2 -0.44454...
6,0 -0.496438 1 -0.465677 2 -0.41583...
7,0 -0.460578 1 -0.369229 2 -0.35411...
8,0 -0.598464 1 -0.559980 2 -0.56824...
9,0 -0.645319 1 -0.584381 2 -0.59375...


In [4]:
!ls ../../../examples/

'Animals Dataset'	   'Gowalla Dataset'  'Vehicles Dataset'
'Datasets overview.ipynb'   README.txt
'Go!Track Dataset'	   'Taxi Dataset'


In [5]:
df = pd.read_csv("../../../examples/Vehicles Dataset/data/vehicles_preapred.zip")

df

,tid,class,t,c1,c2
0,30901,B,0,4207716.0,473841.1
1,30901,B,30,4207724.6,473908.8
2,30901,B,60,4207725.9,473909.6
3,30901,B,90,4207736.9,473915.8
4,30901,B,120,4207763.5,473934.3
...,...,...,...,...,...
178294,420106,B,20700,4205465.6,490337.3
178295,420106,B,20730,4205341.1,490407.3
178296,420106,B,20760,4205316.6,490378.3
178297,420106,B,20790,4205316.6,490378.3


In [6]:
"""df.c1 = 1
df.c2 = 2
df["class"] = 'B'"""

'df.c1 = 1\ndf.c2 = 2\ndf["class"] = \'B\''

In [7]:
def from_2d_array_to_nested(
    X, index=None, columns=None, time_index=None, cells_as_numpy=False
):
    """Convert tabular pandas DataFrame with only primitives in cells into
    nested pandas DataFrame with a single column.
    Parameters
    ----------
    X : pd.DataFrame
    cells_as_numpy : bool, default = False
        If True, then nested cells contain NumPy array
        If False, then nested cells contain pandas Series
    index : array-like, shape=[n_samples], optional (default = None)
        Sample (row) index of transformed DataFrame
    time_index : array-like, shape=[n_obs], optional (default = None)
        Time series index of transformed DataFrame
    Returns
    -------
    Xt : pd.DataFrame
        Transformed DataFrame in nested format
    """

    if (time_index is not None) and cells_as_numpy:
        raise ValueError(
            "`Time_index` cannot be specified when `return_arrays` is True, "
            "time index can only be set to "
            "pandas Series"
        )
    if isinstance(X, pd.DataFrame):
        X = X.to_numpy()

    container = np.array if cells_as_numpy else pd.Series

    # for 2d numpy array, rows represent instances, columns represent time points
    n_instances, n_timepoints = X.shape

    if time_index is None:
        time_index = np.arange(n_timepoints)
    kwargs = {"index": time_index}

    Xt = pd.DataFrame(
        pd.Series([container(X[i, :], **kwargs) for i in range(n_instances)])
    )
    if index is not None:
        Xt.index = index
    if columns is not None:
        Xt.columns = columns
    return Xt

In [8]:
df["pos"] = df.groupby(by=["tid"]).cumcount()

df_pivot_c1 = df.pivot(index=['tid'], columns="pos", values='c1')#.fillna(0)
df_pivot_c2 = df.pivot(index=['tid'], columns="pos", values='c2')#.fillna(0)

df_pivot_c1 = from_2d_array_to_nested(df_pivot_c1)
df_pivot_c2 = from_2d_array_to_nested(df_pivot_c2)

#df_pivot_c1 = df_pivot_c1.fillna(0.0).astype('float64')
#df_pivot_c2 = df_pivot_c2.fillna(0.0).astype('float64')

In [9]:
df_pivot = pd.DataFrame()
df_pivot["dim_0"] = df_pivot_c1[0]
df_pivot["dim_1"] = df_pivot_c2[0]

df_pivot

,dim_0,dim_1
0,0 4207716.0 1 4207724.6 2 42...,0 473841.1 1 473908.8 2 4739...
1,0 4220330.6 1 4220334.0 2 42...,0 486050.8 1 486033.3 2 4860...
2,0 4207981.5 1 4207987.2 2 42...,0 483404.2 1 483284.0 2 4831...
3,0 4207972.5 1 4208076.3 2 42...,0 483463.8 1 483603.6 2 4837...
4,0 4207868.7 1 4207708.4 2 42...,0 478207.5 1 478161.7 2 4780...
...,...,...
376,0 4209018.0 1 4209017.6 2 42...,0 478306.1 1 478460.6 2 4787...
377,0 4209007.4 1 4208991.5 2 42...,0 478322.6 1 478401.6 2 4786...
378,0 4207800.4 1 4207758.1 2 42...,0 483366.9 1 483334.3 2 4831...
379,0 4207814.8 1 4207738.4 2 42...,0 483367.1 1 483241.3 2 4829...


In [10]:
y = df.groupby(by=["tid"]).max()["class"]

In [11]:
df_pivot.to_pickle("df_pivot")
y.to_pickle("y")

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_pivot, y, test_size=0.3)

In [13]:
print(len(X_train) )
len(y_train)

266


266

## Train with MrSQMClassifier

In [14]:
def from_nested_to_2d_array(X, return_numpy=False):
    """Convert nested Panel to 2D numpy Panel.

    Convert nested pandas DataFrame or Series with NumPy arrays or
    pandas Series in cells into tabular
    pandas DataFrame with primitives in cells, i.e. a data frame with the
    same number of rows as the input data and
    as many columns as there are observations in the nested series. Requires
    series to be have the same index.

    Parameters
    ----------
    X : nested pd.DataFrame or nested pd.Series
    return_numpy : bool, default = False
        - If True, returns a NumPy array of the tabular data.
        - If False, returns a pandas DataFrame with row and column names.

    Returns
    -------
     Xt : pandas DataFrame
        Transformed DataFrame in tabular format
    """
    # TODO does not handle dataframes with nested series columns *and*
    #  standard columns containing only primitives

    # convert nested data into tabular data
    if isinstance(X, pd.Series):
        Xt = np.array(X.tolist())

    elif isinstance(X, pd.DataFrame):
        try:
            Xt = np.hstack([X.iloc[:, i].tolist() for i in range(X.shape[1])])

        # except strange key error for specific case
        except KeyError:
            if (X.shape == (1, 1)) and (X.iloc[0, 0].shape == (1,)):
                # in fact only breaks when an additional condition is met,
                # namely that the index of the time series of a single value
                # does not start with 0, e.g. pd.RangeIndex(9, 10) as is the
                # case in forecasting
                Xt = X.iloc[0, 0].values
            else:
                raise

        if Xt.ndim != 2:
            raise ValueError(
                "Tabularization failed, it's possible that not "
                "all series were of equal length"
            )

    else:
        raise ValueError(
            f"Expected input is pandas Series or pandas DataFrame, "
            f"but found: {type(X)}"
        )

    if return_numpy:
        return Xt

    Xt = pd.DataFrame(Xt)

    # create column names from time index
    if X.ndim == 1:
        time_index = (
            X.iloc[0].index
            if hasattr(X.iloc[0], "index")
            else np.arange(X.iloc[0].shape[0])
        )
        columns = [f"{X.name}__{i}" for i in time_index]

    else:
        columns = []
        for colname, col in X.items():
            time_index = (
                col.iloc[0].index
                if hasattr(col.iloc[0], "index")
                else np.arange(col.iloc[0].shape[0])
            )
            columns.extend([f"{colname}__{i}" for i in time_index])

    Xt.index = X.index
    Xt.columns = columns
    return Xt



In [17]:
import warnings
from tqdm.auto import tqdm

from tslearn.piecewise import SymbolicAggregateApproximation

def generateAlphabet(dim):
    return [(97+i).to_bytes(1, byteorder='big')for i in range(dim)] #97 --> 'a'

def test(conf, ts_series):
    ts_series = ts_series.dropna()
    #esempio conf: {'method': 'sax', 'window': 64, 'word': 16, 'alphabet': 3, 'dilation': 1}
    #print(conf, flush=True)

    sax = SymbolicAggregateApproximation(n_segments=conf["word"], alphabet_size_avg=conf["alphabet"], scale=True)
    alphabet = generateAlphabet(conf["alphabet"])
    result = b''
    for i in [0]+list(range(1, len(ts_series)-conf["window"])):
        if i != 0:
            result += b' '
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            X_sax = sax.fit_transform([ts_series.values[i:i+conf["window"]]])
        for char in X_sax[0]:
            result += alphabet[char[0]]

    return result

def MY_transform_time_series(self, ts_x):

        multi_tssr = []

        #ts_x_array = from_nested_to_2d_array(ts_x).values

        if not self.config:
            self.config = []

            min_ws = 16
            min_len = max_len = len(ts_x.iloc[0, 0])
            for a in ts_x.iloc[:, 0]:
                min_len = min(min_len, len(a))
                max_len = max(max_len, len(a))
            max_ws = (min_len + max_len) // 2

            pars = self.create_pars(min_ws, max_ws, self.nsax, random_sampling=True, is_sfa=False)
            for p in pars:
                self.config.append(
                    {'method': 'sax', 'window': p[0], 'word': p[1], 'alphabet': p[2],
                     # 'dilation': np.int32(2 ** np.random.uniform(0, np.log2((min_len - 1) / (p[0] - 1))))})
                     'dilation': 1})

            pars = self.create_pars(min_ws, max_ws, self.nsfa, random_sampling=True, is_sfa=True)
            for p in pars:
                self.config.append(
                    {'method': 'sfa', 'window': p[0], 'word': p[1], 'alphabet': p[2], 'normSFA': False,
                     'normTS': self.sfa_norm
                     })

        for cfg in tqdm(self.config, desc="converting to sax"):
            for i in range(ts_x.shape[1]):
                tssr = []

                if cfg['method'] == 'sax':  # convert time series to SAX
                    #ps = PySAX(cfg['window'], cfg['word'], cfg['alphabet'], cfg['dilation'])
                    for ts in ts_x.iloc[:, i]:
                        sr = test(cfg, ts)#ps.timeseries2SAXseq(ts)
                        if sr == b'': continue
                        tssr.append(sr)
                elif cfg['method'] == 'sfa':
                    pass
                multi_tssr.append(tssr)

        new_multi_tssr = []

        i = 0
        for conf_lat, conf_lon in tqdm(zip(multi_tssr[0::2], multi_tssr[1::2]), desc="converting to trajectory-sax",
                                       total=len(multi_tssr[::2])):
            i += 1
            new_tssr = []
            j = 0

            c = b'a'
            dizionario = {}
            for k in range(len(conf_lat)):  #scorro le varie serie temporali
                #print(f"{i}-{k}, len={len(conf_lat)} {len(conf_lon)}  len_k={len(conf_lat[k])} {len(conf_lon[k])}")
                seq = b''

                for sym1, sym2 in zip(conf_lat[k], conf_lon[k]):
                    j += 1

                    if sym1 == 32:
                        seq += b' '
                        continue

                    key = (sym1, sym2)
                    if key not in dizionario:
                        #print(f"{i}-{k}) {key} not in dict, generating new symbol {str(c)}->{int.from_bytes(c, byteorder='big')+1}")
                        c = (int.from_bytes(c, byteorder='big') + 1).to_bytes(1, byteorder='big')
                        dizionario[key] = c
                    seq += dizionario[key]
                new_tssr.append(seq)
            new_multi_tssr.append(new_tssr)

        return multi_tssr, new_multi_tssr

clf = mrsqm.MrSQMClassifier(MY_transform_time_series, nsax=2, nsfa=0, random_state=42)

try:
    clf.fit(X_train,y_train)
except Exception as e: #raise Exception(X, mr_seqs, self.sequences)
    X = e.args[0]
    mr_seqs_old = e.args[1]
    mr_seqs = e.args[2]
    sequences = e.args[3]
    print("TUTTO OK")

converting to sax:   0%|          | 0/20 [00:00<?, ?it/s]

converting to trajectory-sax:   0%|          | 0/20 [00:00<?, ?it/s]

mining sequences:   0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
mr_seqs[6]

In [ ]:
len(mr_seqs[2])

In [ ]:
matrix = np.zeros(())

In [ ]:
len(X)

In [ ]:
len(y)

In [ ]:
mr_seqs

In [ ]:
mr_seqs_old

In [ ]:
mr_seqs[2]

In [ ]:
mr_seqs_old[0][0]

In [ ]:
clf.config

__mr\_seql__: lista di liste

Ogni lista rappresenta una configurazione (tot: 8*num_dim liste).

Ogni lista di "primo livello" è composta da altre 28 liste (= al numero di istanze)

Queste liste di "secondo livello" contengono un array di byte di lunghezza variabile.
- la lunghezza di questo array dipende dalla TS? (sono tutte lunghe 286 valori)

## Test the model

In [18]:
clf.filters

[SelectKBest(k=500, score_func=<function chi2 at 0x7f25c9cb2e60>),
 SelectKBest(k=500, score_func=<function chi2 at 0x7f25c9cb2e60>),
 SelectKBest(k=500, score_func=<function chi2 at 0x7f25c9cb2e60>),
 SelectKBest(k=500, score_func=<function chi2 at 0x7f25c9cb2e60>),
 SelectKBest(k=500, score_func=<function chi2 at 0x7f25c9cb2e60>),
 SelectKBest(k=500, score_func=<function chi2 at 0x7f25c9cb2e60>),
 SelectKBest(k=500, score_func=<function chi2 at 0x7f25c9cb2e60>),
 SelectKBest(k=500, score_func=<function chi2 at 0x7f25c9cb2e60>),
 SelectKBest(k=500, score_func=<function chi2 at 0x7f25c9cb2e60>),
 SelectKBest(score_func=<function chi2 at 0x7f25c9cb2e60>),
 SelectKBest(score_func=<function chi2 at 0x7f25c9cb2e60>),
 SelectKBest(k=500, score_func=<function chi2 at 0x7f25c9cb2e60>),
 SelectKBest(k=500, score_func=<function chi2 at 0x7f25c9cb2e60>),
 SelectKBest(k=500, score_func=<function chi2 at 0x7f25c9cb2e60>),
 SelectKBest(k=500, score_func=<function chi2 at 0x7f25c9cb2e60>),
 SelectKB

In [19]:
y_pred = clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

converting to sax:   0%|          | 0/20 [00:00<?, ?it/s]

converting to trajectory-sax:   0%|          | 0/20 [00:00<?, ?it/s]

0.8


In [20]:
print(metrics.accuracy_score(y_test, y_pred))

0.8


In [19]:
clf.train_x

array([[False, False, False, ..., False, False,  True],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False,  True, ..., False, False, False],
       [False, False,  True, ..., False, False, False],
       [False, False, False, ..., False, False, False]])